In [1]:
import numpy
import gzip
import numpy as np
import pandas as pd
import sys, glob
import urllib3
import h5py
import cdsapi, zipfile, os, time
import warnings
import shutil
import xarray
from datetime import date
warnings.filterwarnings('ignore')
import pycountry
import json
import copy

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua4 as eua
sys.path.append(os.path.expanduser("~")+'/uvpy/')
import uvfunctions as uvf
import uvplot as uvp

import matplotlib
import matplotlib.pylab as plt
import matplotlib.pyplot as maplt
matplotlib.rcParams.update({'font.size': 20})
matplotlib.rcParams['figure.figsize'] = (20, 10)
font = {'family' : 'normal',
    # 'weight' : 'bold',
    'size'   : 22}
matplotlib.rc('font', **font)

from ipywidgets import IntProgress
from IPython.display import display

In [2]:
with open('/users/staff/uvoggenberger/scratch/hug/config_v9/active.json') as json_file:
    active_v9 = json.load(json_file)
with open('/users/staff/uvoggenberger/scratch/hug/config_v11/active.json') as json_file:
    active_v11 = json.load(json_file)

In [3]:
rq = {'variable': ['air_temperature', 'eastward_wind_speed'], 'date': '19970630-19970701', 'format': 'nc'} 
# 0-20999-0-ZEKG 'date': '19901201-19901201', 'statid': ['0-20999-0-ZSAF','0-20999-0-TTCC'], 'optional': ['wind_bias_estimate',], 'plev':70000, 

In [4]:
df_v9 = eua.vm_request_wrapper(rq , overwrite=True, request_filename='./v9_data/v9.zip', vm_url='http://127.0.0.1:8009')
len(df_v9)

1741

In [22]:
df_v11 = eua.vm_request_wrapper(rq, request_filename='./v11_data/v11.zip')#,overwrite=True)#, vm_url='http://127.0.0.1:8009')
len(df_v11)

1810

In [13]:
try:
    del df_v9
except:
    pass
try:
    del df_v11
except:
    pass

df_v9 = glob.glob('./v9_data/*.nc')
df_v11 = glob.glob('./v11_data/*.nc')
df_v9 = [i.split('/')[-1] for i in df_v9]
df_v11 = [i.split('/')[-1] for i in df_v11]

In [14]:
testlist = df_v9
out = []
for i in df_v11:
    if ~np.isin(i, testlist):
        out.append(i)
        # display(df_v11[i])
        # display(df_v11[i].to_dataframe())
display(out)
print(len(out))

['dest_0-12-20000-60580_air_temperature.nc',
 'dest_0-12-20000-60580_eastward_wind.nc',
 'dest_0-20000-0-04018_air_temperature.nc',
 'dest_0-20000-0-04018_eastward_wind.nc',
 'dest_0-20000-0-12982_air_temperature.nc',
 'dest_0-20000-0-12982_eastward_wind.nc',
 'dest_0-20000-0-17130_air_temperature.nc',
 'dest_0-20000-0-17130_eastward_wind.nc',
 'dest_0-20000-0-26072_air_temperature.nc',
 'dest_0-20000-0-26072_eastward_wind.nc',
 'dest_0-20000-0-47116_eastward_wind.nc',
 'dest_0-20000-0-52652_air_temperature.nc',
 'dest_0-20000-0-52652_eastward_wind.nc',
 'dest_0-20000-0-57245_air_temperature.nc',
 'dest_0-20000-0-57245_eastward_wind.nc',
 'dest_0-20000-0-60252_air_temperature.nc',
 'dest_0-20000-0-60252_eastward_wind.nc',
 'dest_0-20000-0-60461_eastward_wind.nc',
 'dest_0-20000-0-71023_air_temperature.nc',
 'dest_0-20000-0-71023_eastward_wind.nc',
 'dest_0-20000-0-74648_air_temperature.nc',
 'dest_0-20000-0-74648_eastward_wind.nc',
 'dest_0-20000-0-96221_eastward_wind.nc',
 'dest_0-200

90


In [15]:
missing = [('04018', 63.97, 337.4), ('12982', 46.25, 20.1), ('13275', 44.78, 20.53), ('17130', 39.95, 32.88), ('47744', 35.43, 133.35), ('51886', 38.25, 90.85), ('52652', 38.93, 100.43), ('55664', 28.58, 86.62), ('56173', 31.85, 102.7), ('61995', -20.3, 57.5), ('63705', 0.05, 32.45), ('67964', -20.15, 28.62), ('74547', 38.62, 262.7), ('74646', 36.62, 262.52), ('74650', 35.68, 264.13), ('94302', -22.23, 114.08), ('DBBH', 43.5, 310.0), ('ENQT', 37.2, 350.9), ('FNOR', 25.5, 308.0), ('FNRS', 42.8, 344.7), ('JBOA', 31.0, 127.9), ('JCCX', 32.4, 128.1), ('JDWX', 41.2, 141.9), ('JGQH', 18.5, 136.9), ('JIVB', 34.8, 130.0), ('LDWR', 66.0, 2.1), ('OXYH2', 60.2, 352.6), ('UCKZ', -69.4, 74.6)]
for i in missing:
    if i[2] > 180:
        missing.append((i[0], i[1], i[2] - 360))
        missing.remove(i)
display(missing)
len(missing)

[('12982', 46.25, 20.1),
 ('13275', 44.78, 20.53),
 ('17130', 39.95, 32.88),
 ('47744', 35.43, 133.35),
 ('51886', 38.25, 90.85),
 ('52652', 38.93, 100.43),
 ('55664', 28.58, 86.62),
 ('56173', 31.85, 102.7),
 ('61995', -20.3, 57.5),
 ('63705', 0.05, 32.45),
 ('67964', -20.15, 28.62),
 ('74646', 36.62, 262.52),
 ('94302', -22.23, 114.08),
 ('ENQT', 37.2, 350.9),
 ('FNRS', 42.8, 344.7),
 ('JBOA', 31.0, 127.9),
 ('JCCX', 32.4, 128.1),
 ('JDWX', 41.2, 141.9),
 ('JGQH', 18.5, 136.9),
 ('JIVB', 34.8, 130.0),
 ('LDWR', 66.0, 2.1),
 ('UCKZ', -69.4, 74.6),
 ('04018', 63.97, -22.600000000000023),
 ('74547', 38.62, -97.30000000000001),
 ('74650', 35.68, -95.87),
 ('DBBH', 43.5, -50.0),
 ('FNOR', 25.5, -52.0),
 ('OXYH2', 60.2, -7.399999999999977)]

28

In [16]:
# check by name:
print('searching by name: ')
not_in_diff = []
in_diff = []
print('not found:')
for i in missing:
    if np.invert(i[0] in '\t'.join(df_v11)):
        not_in_diff.append(i)
        print(i)
    else:
        in_diff.append(i)

print(len(not_in_diff))
print()
print('found:')
display(in_diff)
print(len(in_diff))

searching by name: 
not found:
('47744', 35.43, 133.35)
('51886', 38.25, 90.85)
('55664', 28.58, 86.62)
('74646', 36.62, 262.52)
('FNRS', 42.8, 344.7)
('74547', 38.62, -97.30000000000001)
('74650', 35.68, -95.87)
('OXYH2', 60.2, -7.399999999999977)
8

found:


[('12982', 46.25, 20.1),
 ('13275', 44.78, 20.53),
 ('17130', 39.95, 32.88),
 ('52652', 38.93, 100.43),
 ('56173', 31.85, 102.7),
 ('61995', -20.3, 57.5),
 ('63705', 0.05, 32.45),
 ('67964', -20.15, 28.62),
 ('94302', -22.23, 114.08),
 ('ENQT', 37.2, 350.9),
 ('JBOA', 31.0, 127.9),
 ('JCCX', 32.4, 128.1),
 ('JDWX', 41.2, 141.9),
 ('JGQH', 18.5, 136.9),
 ('JIVB', 34.8, 130.0),
 ('LDWR', 66.0, 2.1),
 ('UCKZ', -69.4, 74.6),
 ('04018', 63.97, -22.600000000000023),
 ('DBBH', 43.5, -50.0),
 ('FNOR', 25.5, -52.0)]

20


In [21]:
# check by lat/lon threshold 1°:
print('searching by lat/lon: ')
remaining = copy.deepcopy(missing)
found = []
found_where = []
max_count = len(df_v11)
f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

for i in df_v11:
    with h5py.File('./v11_data/'+i) as file:
        for j in missing:
            if np.logical_and(np.nanmin(np.abs(np.array(file['lat'][:]) - j[1])) < 1, np.nanmin(np.abs(np.array(file['lon'][:]) - j[2])) < 1):
                found.append(j)
                found_where.append((i,np.nanmin(np.abs(np.array(file['lat'][:]) - j[1])), np.nanmin(np.abs(np.array(file['lon'][:]) - j[2]))))
                try:
                    remaining.remove(j)
                except:
                    pass
    f.value += 1
print('not found:')
display(remaining)
print(len(remaining))
print()
print('found')
display(found)
print(len(found))
display(found_where)

searching by lat/lon: 


IntProgress(value=0, max=1810)

not found:


[('51886', 38.25, 90.85),
 ('55664', 28.58, 86.62),
 ('74646', 36.62, 262.52),
 ('ENQT', 37.2, 350.9),
 ('FNRS', 42.8, 344.7),
 ('74547', 38.62, -97.30000000000001),
 ('OXYH2', 60.2, -7.399999999999977)]

7

found


[('04018', 63.97, -22.600000000000023),
 ('04018', 63.97, -22.600000000000023),
 ('12982', 46.25, 20.1),
 ('12982', 46.25, 20.1),
 ('13275', 44.78, 20.53),
 ('13275', 44.78, 20.53),
 ('17130', 39.95, 32.88),
 ('17130', 39.95, 32.88),
 ('JDWX', 41.2, 141.9),
 ('JDWX', 41.2, 141.9),
 ('47744', 35.43, 133.35),
 ('47744', 35.43, 133.35),
 ('52652', 38.93, 100.43),
 ('52652', 38.93, 100.43),
 ('61995', -20.3, 57.5),
 ('61995', -20.3, 57.5),
 ('63705', 0.05, 32.45),
 ('67964', -20.15, 28.62),
 ('67964', -20.15, 28.62),
 ('74650', 35.68, -95.87),
 ('74650', 35.68, -95.87),
 ('94302', -22.23, 114.08),
 ('94302', -22.23, 114.08),
 ('56173', 31.85, 102.7),
 ('56173', 31.85, 102.7),
 ('DBBH', 43.5, -50.0),
 ('DBBH', 43.5, -50.0),
 ('FNOR', 25.5, -52.0),
 ('FNOR', 25.5, -52.0),
 ('JBOA', 31.0, 127.9),
 ('JBOA', 31.0, 127.9),
 ('JCCX', 32.4, 128.1),
 ('JCCX', 32.4, 128.1),
 ('JDWX', 41.2, 141.9),
 ('JDWX', 41.2, 141.9),
 ('JGQH', 18.5, 136.9),
 ('JGQH', 18.5, 136.9),
 ('JIVB', 34.8, 130.0),
 ('JIVB

43


[('dest_0-20000-0-04018_air_temperature.nc', 0.0, 0.0),
 ('dest_0-20000-0-04018_eastward_wind.nc', 0.0, 0.0),
 ('dest_0-20000-0-12982_air_temperature.nc', 0.0, 0.0),
 ('dest_0-20000-0-12982_eastward_wind.nc', 0.0, 0.0),
 ('dest_0-20000-0-13275_air_temperature.nc', 0.0099983215, 0.11000061),
 ('dest_0-20000-0-13275_eastward_wind.nc', 0.0099983215, 0.11000061),
 ('dest_0-20000-0-17130_air_temperature.nc', 0.0, 0.0),
 ('dest_0-20000-0-17130_eastward_wind.nc', 0.0, 0.0),
 ('dest_0-20000-0-47580_air_temperature.nc', 0.5, 0.519989),
 ('dest_0-20000-0-47580_eastward_wind.nc', 0.5, 0.519989),
 ('dest_0-20000-0-47741_air_temperature.nc', 0.0, 1.5258789e-05),
 ('dest_0-20000-0-47741_eastward_wind.nc', 0.0, 1.5258789e-05),
 ('dest_0-20000-0-52652_air_temperature.nc', 0.0, 0.0),
 ('dest_0-20000-0-52652_eastward_wind.nc', 0.0, 0.0),
 ('dest_0-20000-0-61995_air_temperature.nc', 0.0, 0.0),
 ('dest_0-20000-0-61995_eastward_wind.nc', 0.0, 0.0),
 ('dest_0-20000-0-63705_eastward_wind.nc', 0.0, 0.0),
 ('d